In [1]:
# クラスタリング

# 目的：関連のある文章を見つける

# 前章まで => 教師あり学習
#     * ラベル付きデータの取得が困難な場合もある
#     * 人がラベルを付けるのは非常にコスト

# 教師なし学習
#     * データだけからパターンを見つけ出すことが可能

# 例
#     * Q&Aサイトを考える
#     * ユーザが探している情報（キーワード）を検索すると、ふさわしいページを表示する
#     * 探している回答でない場合、関連した質問を表示したい

# 単純なアプローチ
#     * すべての類似度を算出 => コストが非常に高い

# クラスタリング
#      * クラスタ => データの集合
#      * 似ているものは同じクラスタ、似てないものは別のクラスタに
#      * 今回の場合、テキストデータ同士で類似度を算出する方法が必要
#      * SciKit ライブラリを利用する

In [2]:
# 3.1 文書の関連性を計測する

# テキストデータのままでは計測は困難
# テキストデータを数字（ベクトル）に変換することが必要 => 類似度の計算が可能

In [3]:
# 3.1.1 やってはいけないこと

# 類似度を求める手法 => レーベンシュタイン距離（編集距離）
# レーベンシュタイン距離
#      * 編集を行う最小回数
#      * 「machine」と「mchiene」 => 距離は 2
#      * これを求める計算コストは高い
#      * 単語レベルの編集距離は考えるべきでない

# 応用
#     * 文章全体で編集距離を計測（単語ごと）
#     * 「How to format my hard disk」と「Hard disk format problems」 => 距離 6
#     * 文字レベルより早いが、依然として時間かかる


In [4]:
# 3.1.2 どうやるべきか

# bag-of-words
#     * 単語の出現回数を特徴量として用いる => 結果をベクトル化
#     * ベクトルのサイズが大きくなりがち
#     * ベクトル化することでユーグリット距離の計算が可能 => 最近傍点を発見
#     * 時間がかかる

# 手順
#     1. 各文書から特徴量を抽出し、ベクトルの形で保存
#     2. 特徴ベクトルに対してクラスタリングを行う
#     3. 投稿された質問書に対して、クラスタを決定
#     4. このクラスタに属する文書を他に集め、多様性を増す

In [248]:
# 3.2 前処理

# 共通する単語の出現回数を類似度として計測する

In [250]:
# 3.2.1 テキストデータを bag-of-words に変換する

# ライブラリをインポートするだけ

from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(min_df=1)
print(vectorizer)

# パラメータ min_dif : 頻繁には使われていない単語を無視する（ここでは一回未満）
# print することで、デフォルトのパラメータを表示

# analyzer='word'  : 単語レベルで出現回数をカウント
# token_pattern : 単語の決定方法を正規表現

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)


In [251]:
content = ["How to format my hard disk", "Hard disk format problems"]
X = vectorizer.fit_transform(content)

# 数えるべき単語を抜き出す
print(vectorizer.get_feature_names())

# それぞれの文書が数えるべき単語を何個含んでいるかを表すベクトル
print(X.toarray().transpose())

['disk', 'format', 'hard', 'how', 'my', 'problems', 'to']
[[1 1]
 [1 1]
 [1 1]
 [1 0]
 [1 0]
 [0 1]
 [1 0]]


In [252]:
# 3.2.2 単語を数える

import os

from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(min_df=1)

DIR = './data/toys'

# DIR 以下にある txt ファイルを読み込む
posts = [open(os.path.join(DIR, f)).read() for f in sorted(os.listdir(DIR))]

for post in posts:
    print(post)

This is a toy post about machine learning. Actually, it contains not much interesting stuff.
Imaging databases provide storage capabilities.
Most imaging databases save images permanently.

Imaging databases store data.
Imaging databases store data. Imaging databases store data. Imaging databases store data.


In [253]:
# vectorizer のパラメータに従い、ベクトル化
X_train = vectorizer.fit_transform(posts)

# サンプル数と、数えるべき単語数を取得
num_samples, num_features = X_train.shape
print("#samples: %d, #features: %d" % (num_samples, num_features))

#samples: 5, #features: 25


In [190]:
# 数えるべき単語のラベルを取得
print(vectorizer.get_feature_names())

['about', 'actually', 'capabilities', 'contains', 'data', 'databases', 'images', 'imaging', 'interesting', 'is', 'it', 'learning', 'machine', 'most', 'much', 'not', 'permanently', 'post', 'provide', 'save', 'storage', 'store', 'stuff', 'this', 'toy']


In [254]:
# 新しい文書をベクトル化
new_post = "imaging databases"
new_post_vec = vectorizer.transform([new_post])

In [255]:
# ほとんどの要素が 0 のベクトルになる
# そのため、出現した単語のみの情報だけ格納する
print(new_post_vec)
print()

# toarray() を用いることで、特徴ベクトルのすべての要素を表示できる（サンプル数が増えると膨大になる）
print(new_post_vec.toarray())
print()

# 特徴ベクトルの長さ
print(len(new_post_vec.toarray()[0]))

  (0, 5)	1
  (0, 7)	1

[[0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]

25


In [193]:
# 類似度を求める
# 簡単な手法として、ユーグリット距離を用いる
# 計算するためには、toarray() を用いて、特徴ベクトルの形式にする
# ユーグリットノムル（ユーグリット距離）は scipy の norm 関数で計算できる

import scipy as sp
def dist_raw(v1, v2):
    delta = v1 - v2
    return sp.linalg.norm(delta.toarray())


In [256]:
# 実際に計算してみる
# 

import sys

best_doc = None
best_dist = sys.maxsize
best_i = None

# サンプル数（入力する文書数）回ループ
for i in range(0, num_samples):
    post = posts[i]
    if post == new_post:
        continue
    # 特徴ベクトルから、出現した単語のみの情報だけ格納する
    post_vec = X_train.getrow(i)
    # print(post_vec)
    
    # new_post とのユーグリット距離を計算
    d = dist_raw(post_vec, new_post_vec)

    print("=== Post %i with dist=%.2f: %s" % (i, d, post))

    # 類似度が高い場合値を更新
    if d < best_dist:
        best_dist = d
        best_i = i

# 結果
print("Best post is %i with dist=%.2f" % (best_i, best_dist))

# 考察
# 文書0 が最も似ていない => 共通する単語なし　（合ってる？）
# 文書1 と文書3 は共に新しい文書の単語を含み、類似度も高い。しかし、文書3 のが文書が短いため、最も類似していると言える
# 文書3,4 は繰り返してるだけ。本来同じ類似度なはずだけど…

=== Post 0 with dist=4.00: This is a toy post about machine learning. Actually, it contains not much interesting stuff.
=== Post 1 with dist=1.73: Imaging databases provide storage capabilities.
=== Post 2 with dist=2.00: Most imaging databases save images permanently.

=== Post 3 with dist=1.41: Imaging databases store data.
=== Post 4 with dist=5.10: Imaging databases store data. Imaging databases store data. Imaging databases store data.
Best post is 3 with dist=1.41


In [257]:
# 出現回数だけ見ると、ユーグリット距離が大きくなってしまう
# => これだけでは単純すぎ
# 特徴ベクトルの単位長さにするために正規化が必要
print(X_train.getrow(3).toarray())
print(X_train.getrow(4).toarray())

[[0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]]
[[0 0 0 0 3 3 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0]]


In [228]:
# 3.2.3 単語の出現回数ベクトルを正規化する

def dist_norm(v1, v2):
    v1_normalized = v1 / sp.linalg.norm(v1.toarray())
    v2_normalized = v2 / sp.linalg.norm(v2.toarray())
    
#     print(v1_normalized)
#     print(v2_normalized)

    delta = v1_normalized - v2_normalized

    return sp.linalg.norm(delta.toarray())

In [258]:
# 正規化して計算してみる

best_doc = None
best_dist = sys.maxsize
best_i = None

# サンプル数（入力する文書数）回ループ
for i in range(0, num_samples):
    post = posts[i]
    if post == new_post:
        continue
    # 特徴ベクトルから、出現した単語のみの情報だけ格納する
    post_vec = X_train.getrow(i)
    
    # new_post とのユーグリット距離を計算
    d = dist_norm(post_vec, new_post_vec)

    print("=== Post %i with dist=%.2f: %s" % (i, d, post))

    # 類似度が高い場合値を更新
    if d < best_dist:
        best_dist = d
        best_i = i

# 結果
print("Best post is %i with dist=%.2f" % (best_i, best_dist))

# 考察


=== Post 0 with dist=1.41: This is a toy post about machine learning. Actually, it contains not much interesting stuff.
=== Post 1 with dist=0.86: Imaging databases provide storage capabilities.
=== Post 2 with dist=0.92: Most imaging databases save images permanently.

=== Post 3 with dist=0.77: Imaging databases store data.
=== Post 4 with dist=0.77: Imaging databases store data. Imaging databases store data. Imaging databases store data.
Best post is 3 with dist=0.77


In [260]:
# 3.2.4 重要度の低い単語を取り除く

# ストップワード（多くの文書に出現し、情報をあまり持たない単語）を設定する
vectorizer = CountVectorizer(min_df=1, stop_words='english')

# ストップワード例（一部）
print(sorted(vectorizer.get_stop_words())[0:20])

['a', 'about', 'above', 'across', 'after', 'afterwards', 'again', 'against', 'all', 'almost', 'alone', 'along', 'already', 'also', 'although', 'always', 'am', 'among', 'amongst', 'amoungst']


In [261]:
# ストップワードを用いると、単語リストが 7 つ減った

# vectorizer のパラメータに従い、ベクトル化
X_train = vectorizer.fit_transform(posts)

# サンプル数と、数えるべき単語数を取得
num_samples, num_features = X_train.shape
print("#samples: %d, #features: %d" % (num_samples, num_features))

# 新しい文書をベクトル化
new_post = "imaging databases"
new_post_vec = vectorizer.transform([new_post])

#samples: 5, #features: 18


In [262]:
# 同様に、類似度を計算

best_doc = None
best_dist = sys.maxsize
best_i = None

# サンプル数（入力する文書数）回ループ
for i in range(0, num_samples):
    post = posts[i]
    if post == new_post:
        continue
    # 特徴ベクトルから、出現した単語のみの情報だけ格納する
    post_vec = X_train.getrow(i)
    
    # new_post とのユーグリット距離を計算
    d = dist_norm(post_vec, new_post_vec)

    print("=== Post %i with dist=%.2f: %s" % (i, d, post))

    # 類似度が高い場合値を更新
    if d < best_dist:
        best_dist = d
        best_i = i

# 結果
print("Best post is %i with dist=%.2f" % (best_i, best_dist))

# 考察
# ここではあまり大きな変化は見られなかったが、重要
# 

=== Post 0 with dist=1.41: This is a toy post about machine learning. Actually, it contains not much interesting stuff.
=== Post 1 with dist=0.86: Imaging databases provide storage capabilities.
=== Post 2 with dist=0.86: Most imaging databases save images permanently.

=== Post 3 with dist=0.77: Imaging databases store data.
=== Post 4 with dist=0.77: Imaging databases store data. Imaging databases store data. Imaging databases store data.
Best post is 3 with dist=0.77


In [263]:
# 3.2.5 ステミング

# 意味的には同じ単語が語形変形により異なる形でカウントされてしまう
# imaging と　images は同じ単語としてカウントするほうが理にかなっている
# NLTK (Natural Language Toolkit) を用いることで、CountVectorizer のプラグインとして実現可能（日本語ver 要調査）

import nltk

In [266]:
# ステミング例

s = nltk.stem.SnowballStemmer('english')
print(s.stem("graphics"))
print(s.stem('imaging'))
print(s.stem('image'))
print(s.stem('imagination'))
print(s.stem('imagine'))

graphic
imag
imag
imagin
imagin


In [267]:
# ステミングは必ずしも正しいとは限らない

print(s.stem('buys'))
print(s.stem('buying'))
print(s.stem('bought'))

buy
buy
bought


In [268]:
# NLTK のステマーを用いてベクトル化を拡張する

from sklearn.feature_extraction.text import CountVectorizer
import nltk.stem
english_stemmer = nltk.stem.SnowballStemmer('english')

class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        # これよくわからない
        # build_analyzer 調べる
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        # それぞれの単語をステマーして格納
        return lambda doc: (english_stemmer.stem(w) for w in analyzer(doc))
        
vectorizer = StemmedCountVectorizer(min_df=1, stop_words='english')

# vectorizer のパラメータに従い、ベクトル化
X_train = vectorizer.fit_transform(posts)

# サンプル数と、数えるべき単語数を取得
num_samples, num_features = X_train.shape
print("#samples: %d, #features: %d" % (num_samples, num_features))

# 新しい文書をベクトル化
new_post = "imaging databases"
new_post_vec = vectorizer.transform([new_post])

#samples: 5, #features: 17


In [269]:
# 同様に類似度を求める

best_doc = None
best_dist = sys.maxsize
best_i = None

# サンプル数（入力する文書数）回ループ
for i in range(0, num_samples):
    post = posts[i]
    if post == new_post:
        continue
    # 特徴ベクトルから、出現した単語のみの情報だけ格納する
    post_vec = X_train.getrow(i)
    
    # new_post とのユーグリット距離を計算
    d = dist_norm(post_vec, new_post_vec)

    print("=== Post %i with dist=%.2f: %s" % (i, d, post))

    # 類似度が高い場合値を更新
    if d < best_dist:
        best_dist = d
        best_i = i

# 結果
print("Best post is %i with dist=%.2f" % (best_i, best_dist))

# 考察
# 文書2 が image を２つ含むようになるため、最も類似度が高くなった

=== Post 0 with dist=1.41: This is a toy post about machine learning. Actually, it contains not much interesting stuff.
=== Post 1 with dist=0.86: Imaging databases provide storage capabilities.
=== Post 2 with dist=0.63: Most imaging databases save images permanently.

=== Post 3 with dist=0.77: Imaging databases store data.
=== Post 4 with dist=0.77: Imaging databases store data. Imaging databases store data. Imaging databases store data.
Best post is 2 with dist=0.63


In [270]:
# 3.2.6 TF-IDF を用いる

# これまで：　出現頻度 => 重要度
# どの文書にも存在するような単語も存在するため、十分でない
# max_df=0.9 とすることで、すべての文書の 90 % 以上に出現する単語を除外できるが、89 % の場合は？
# max_df をどのように設定すればいいか問題

# TF-IDF
# 特定の文書だけで現れやすい単語、つまり、他の文書ではあまり現れない単語は、その文書において特徴量が大きい

# 手法
# ➀ ある単語に対して、対象の文書中で出現した回数をカウント
# ➁ その単語が他の文書でどれだけ出現するかをカウント
# ➂ ➀ / ➁
# 
#  多く出現するほど値は大きくなる　かつ　他の文書で多く出現すると値は小さくなる

In [241]:
# 実装

import math
import scipy as sp

def tfidf(term, doc, docset):
    tf = float(doc.count(term) / sum(doc.count(w) for w in set(doc)))
    idf = math.log(float(len(docset)) / (len([doc for doc in docset if term in doc])))
    return tf * idf

In [272]:
# test

doc_a, doc_abb, doc_abc = ["a"], ["a", "b", "b"], ["a", "b", "c"]
D = [doc_a, doc_abb, doc_abc]
print(tfidf("a", doc_a, D))
print(tfidf("b", doc_abb, D))
print(tfidf("a", doc_abc, D))
print(tfidf("b", doc_abc, D))
print(tfidf("c", doc_abc, D))

0.0
0.27031007207210955
0.0
0.13515503603605478
0.3662040962227032


In [273]:
# a はすべての文書で用いられるため、意味を持たない
# b は doc_abb の文書にとって 、doc_abc の２倍重要
# c は doc_abc にとって最も特徴量が大きい単語

In [274]:
# TF-IDF も scikitlearn で利用可能
# TfidfVectorizer 　(CountVectorizer を継承したクラス) に含まれる

from sklearn.feature_extraction.text import TfidfVectorizer

class StemmedTfidfVectorizer(TfidfVectorizer):
    def build_analyzer(self):
        analyzer = super(TfidfVectorizer, self).build_analyzer()
        return lambda doc: (english_stemmer.stem(w) for w in analyzer(doc))
        
vectorizer = StemmedTfidfVectorizer(min_df=1, stop_words='english')

# vectorizer のパラメータに従い、ベクトル化
X_train = vectorizer.fit_transform(posts)

# サンプル数と、数えるべき単語数を取得
num_samples, num_features = X_train.shape
print("#samples: %d, #features: %d" % (num_samples, num_features))

# 新しい文書をベクトル化
new_post = "imaging databases"
new_post_vec = vectorizer.transform([new_post])

#samples: 5, #features: 17


In [275]:
# 同様

best_doc = None
best_dist = sys.maxsize
best_i = None

# サンプル数（入力する文書数）回ループ
for i in range(0, num_samples):
    post = posts[i]
    if post == new_post:
        continue
    # 特徴ベクトルから、出現した単語のみの情報だけ格納する
    post_vec = X_train.getrow(i)
    
    # new_post とのユーグリット距離を計算
    d = dist_norm(post_vec, new_post_vec)

    print("=== Post %i with dist=%.2f: %s" % (i, d, post))

    # 類似度が高い場合値を更新
    if d < best_dist:
        best_dist = d
        best_i = i

# 結果
print("Best post is %i with dist=%.2f" % (best_i, best_dist))


=== Post 0 with dist=1.41: This is a toy post about machine learning. Actually, it contains not much interesting stuff.
=== Post 1 with dist=1.08: Imaging databases provide storage capabilities.
=== Post 2 with dist=0.86: Most imaging databases save images permanently.

=== Post 3 with dist=0.92: Imaging databases store data.
=== Post 4 with dist=0.92: Imaging databases store data. Imaging databases store data. Imaging databases store data.
Best post is 2 with dist=0.86


In [246]:
# 3.2.7 テキストデータの前処理としてここまでやってきたこと

# 1. テキストデータをトークン化
# 2. 頻出しすぎる単語は、関連する文書を見つけるために役に立たないため除外
# 3. めったに使われない単語は、新しい文書でも使われる可能性が低いため除外
# 4. 残った単語について、その出現回数をカウント
# 5. 文書全体の状況を考慮するため、単語の出現回数から TF-IDF を計算

# ノイズのあるテキストデータから、簡潔にまとめられた特徴量を得ることが可能

# bag-of-words は単純だが、優れた性能（ただし、欠点も多い）
# 欠点
# 「Car hits wall」 と　「Wall hits car」 は同じ特徴ベクトル
# 「I will eat ice cream」 と　「I will not eat ice cream」 の特徴ベクトルは意味は逆だが非常に近い
# （上は、２つの単語のペアや３つの単語のペアを一つのまとまりとしてカウントすることで解決可能）
# タイポには対応不可

# ここまでで、効率的にクラスタリングを行う用意が完了